In [ ]:
import requests
import json
from pathlib import Path
from tqdm import tqdm

In [ ]:
def download(datasets):
    downloaded = []
    
    for d in tqdm(datasets):

        orga = d['organization']['slug'] if d['organization'] else d['owner']['slug']
        slug = d['slug']
        id = d['id']
        
        for r in d['resources']:
            rurl = r['url']
            rid = r['id']

            # ODS style NB: won't work more than once for CKAN
            if 'format=csv' in rurl:
                filename = rurl.split('/')[-3] + '.csv'
            else:
                filename = rurl.split('/')[-1]
            
            if filename in downloaded:
                print('x existing file %s' % rurl)
                continue
            
            ext = filename.split('.')[-1]
            
            if ext not in ['csv', 'xls', 'xlsx']:
                print('x ignored file %s' % rurl)
                continue
            
            r = requests.get(rurl, allow_redirects=True)
            p = Path('out/data/%s' % id)
            
            p.mkdir(exist_ok=True)
            
            #l = 253 -len(ext) -1
            #rid = rid[:l]
            written_filename = '%s.%s' % (rid, ext) 
            
            with open('%s/%s' % (p, written_filename), 'wb') as f:
                f.write(r.content)
                downloaded.append(filename)
                #print('- downloaded file [%s] %s' % (filename, rurl))
    
download(json.load(open("out/datasets.json")))